In [1]:
!python --version

Python 3.7.3


In [3]:
!pip install -q -U pip --ignore-installed wrapt enum34 simplejson netaddr
!pip install -q numpy
!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets

In [4]:
import pandas as pd
import numpy as np

from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [5]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [6]:
df_train

,index,class,title
0,0,體育,亞洲杯奪冠賠率：日本、伊朗領銜 中國竟與泰國並列
1,1,體育,9輪4球本土射手僅次武磊 黃紫昌要搶最強U23頭銜
2,2,體育,如果今年勇士奪冠，下賽季詹姆斯何去何從？
3,3,體育,超級替補！科斯塔本賽季替補出場貢獻7次助攻
4,4,體育,騎士6天里發生了啥？從首輪搶七到次輪3-0猛龍
5,5,體育,如果朗多進入轉會市場，哪些球隊適合他？
6,6,體育,詹姆斯G3決殺，你怎麼看？
7,7,體育,大魔王帶頭唱歌！火箭這像是打季後賽？爵士神帥這話已提前投降了
8,8,體育,馬夏爾要去切爾西？可以商量，不過穆里尼奧的要價是4000萬加威廉
9,9,體育,利希施泰納宣佈賽季結束後離隊：我需要新的挑戰


In [7]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [29]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [9]:
ws = WS('./data/')
pos = POS('./data/')

In [13]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    # YOUR CODE HERE

    # END YOUR CODE
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [14]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    # YOUR CODE HERE

    # END YOUR CODE
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [11]:
train_title_cuts[120]

[('國腳', 'Na'),
 ('張呈棟', 'Nb'),
 ('：', 'COLONCATEGORY'),
 ('從', 'D'),
 ('沒', 'D'),
 ('想', 'VE'),
 ('過', 'Di'),
 ('自己', 'Nh'),
 ('會', 'D'),
 ('出', 'VC'),
 ('一', 'Neu'),
 ('本', 'Nf'),
 ('書', 'Na')]

# Bag of Words (BOW)

In [16]:
word2index = {}
index2word = {}
# 產生字與index對應的關係
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word not in word2index:
            word2index[word] = n 
            index2word[n] = word
            n += 1

In [17]:
word2index['溫暖']

1520

In [18]:
index2word[1520]

'溫暖'

In [19]:
def get_bow_vector(pairs, word2index):
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index:
            vector[word2index[word]] += 1
    return vector

In [20]:
get_bow_vector(train_title_cuts[120], word2index)

array([0., 0., 0., ..., 0., 0., 0.])

# 排除較無意義的詞性

In [21]:
pos_analysis = {}
for _, pairs in train_title_cuts.items():
    for word, flag in pairs:
        if flag not in pos_analysis:
            pos_analysis[flag] = set()
        pos_analysis[flag].add(word)

for flag, words in pos_analysis.items():
    print(flag, ':', list(words)[:100])
    print('=======================')

Nb : ['林美景', '尤納斯', '賞月夜', '朱婷', '伏爾加格勒', '阿迪薩亞', '賈躍亭', '浦發', '億航', '重渡溝', '傑森', '羅', '內馬爾', '清華', '李白界', '郭煒煒', '李秋平', '娜迦王海伯倫', '韓信', '埃爾克森', '杜蘭特', '微博', '亨德森', '圍甲', '雷軍', '國四', '聯盟里', '萊萬特', '阿木木歷', '侯森', '北京人和', '華峰', '大豐', '英超', '火箭隊', '迪士尼', '郎平', '逆水寒', '扎堆', '約翰塞納', '詹姆斯', '萊科寧', '山東隊', '西蒙斯', '皇馬', '上港', '胤祥', '吉尼斯', '溫格', '力帆', '泰森', '高曉松', '楊帆', '哪吒', '馬夏爾', '老詹', '萬孚', '奧拉朱旺', '瓦基弗', '阿里舜宇', '張呈棟', '靜懿', '卡拉黑', '德羅贊', '雷霆隊', '許', '盧', '藍星', '郭躍', '張大仙', '璞麗', '唐培科', '劉偉', '意大利杯', '施之皓', '昂科威', '堪比比薩', '權健', '詹', '海爾', '林斌', '裴擒虎', '威廉姆斯', '沙特', '劉集', '英特爾', '雲聯惠', '老馬', '拉基蒂奇', '樸泰夏', '威少', '劉德華', '東決', '小明', '微星', '巴菲特', '施泰納', '洛瑞', '遼足', '李嘉誠']
VC : ['賣出', '丟', '開展', '撫摸', '耍', '練到', '縊縮', '翻譯', '救贖', '攻', '帶', '打開', '參透', '融入', '外接', '辦', '應用', '招待', '檢驗', '啓定', '迎接', '幫助', '採摘', '賒', '提升', '撒', '精選', '考察', '擴種', '反擊', '參加', '包裝', '毀滅', '養成', '執行', '回應', '牽頭', '採取', '宣一', '發推', '抓', '取消', '戰釀', '照看', '論', '看', '打通', '玩', '踢到', '擺出', '娶', '監管', '防守', '通過', '單殺', '接近',

|         Type        |     Description    |
|:-------------------:|:------------------:|
| A                   | 非謂形容詞         |
| Caa                 | 對等連接詞         |
| Cab                 | 連接詞，如：等等   |
| Cba                 | 連接詞，如：的話   |
| Cbb                 | 關聯連接詞         |
| D                   | 副詞               |
| Da                  | 數量副詞           |
| Dfa                 | 動詞前程度副詞     |
| Dfb                 | 動詞後程度副詞     |
| Di                  | 時態標記           |
| Dk                  | 句副詞             |
| DM                  | 定量式             |
| I                   | 感嘆詞             |
| Na                  | 普通名詞           |
| Nb                  | 專有名詞           |
| Nc                  | 地方詞             |
| Ncd                 | 位置詞             |
| Nd                  | 時間詞             |
| Nep                 | 指代定詞           |
| Neqa                | 數量定詞           |
| Neqb                | 後置數量定詞       |
| Nes                 | 特指定詞           |
| Neu                 | 數詞定詞           |
| Nf                  | 量詞               |
| Ng                  | 後置詞             |
| Nh                  | 代名詞             |
| Nv                  | 名物化動詞         |
| P                   | 介詞               |
| T                   | 語助詞             |
| VA                  | 動作不及物動詞     |
| VAC                 | 動作使動動詞       |
| VB                  | 動作類及物動詞     |
| VC                  | 動作及物動詞       |
| VCL                 | 動作接地方賓語動詞 |
| VD                  | 雙賓動詞           |
| VF                  | 動作謂賓動詞       |
| VE                  | 動作句賓動詞       |
| VG                  | 分類動詞           |
| VH                  | 狀態不及物動詞     |
| VHC                 | 狀態使動動詞       |
| VI                  | 狀態類及物動詞     |
| VJ                  | 狀態及物動詞       |
| VK                  | 狀態句賓動詞       |
| VL                  | 狀態謂賓動詞       |
| V_2                 | 有                 |
|                     |                    |
| DE                  | 的之得地           |
| SHI                 | 是                 |
| FW                  | 外文               |
|                     |                    |
| COLONCATEGORY       | 冒號               |
| COMMACATEGORY       | 逗號               |
| DASHCATEGORY        | 破折號             |
| DOTCATEGORY         | 點號               |
| ETCCATEGORY         | 刪節號             |
| EXCLAMATIONCATEGORY | 驚嘆號             |
| PARENTHESISCATEGORY | 括號               |
| PAUSECATEGORY       | 頓號               |
| PERIODCATEGORY      | 句號               |
| QUESTIONCATEGORY    | 問號               |
| SEMICOLONCATEGORY   | 分號               |
| SPCHANGECATEGORY    | 雙直線             |
| WHITESPACE          | 空白               |

In [22]:
def get_bow_vector_with_selection(pairs, word2index):
    excluded_flags = [
        'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
        'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
        'Caa', 'Cab', 'Cba', 'Cbb',
        'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
        'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
        'SPCHANGECATEGORY', 'WHITESPACE'
    ]
    vector = np.zeros(len(word2index))
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            vector[word2index[word]] += 1
    return vector

# Cosine Similarity

In [23]:
def cosine_similarity(bow1, bow2):
    similarity = np.sum(bow1 * bow2) / (np.sqrt(np.sum(np.square(bow1))) * np.sqrt(np.sum(np.square(bow2))))
    return similarity

In [24]:
bow1 = get_bow_vector(train_title_cuts[100], word2index)
bow2 = get_bow_vector(train_title_cuts[130], word2index)
cosine_similarity(bow1, bow2)

0.08703882797784893

In [25]:
train_title_cuts[100]

[('山東', 'Nc'),
 ('魯能', 'Nb'),
 ('有沒有', 'D'),
 ('可能', 'D'),
 ('拿到', 'VC'),
 ('今年', 'Nd'),
 ('的', 'DE'),
 ('中', 'A'),
 ('超', 'A'),
 ('冠軍', 'Na'),
 ('？', 'QUESTIONCATEGORY')]

In [26]:
train_title_cuts[130]

[('NBA', 'Nb'),
 ('和', 'Caa'),
 ('CBA', 'FW'),
 ('差距', 'Na'),
 ('在', 'P'),
 ('哪裡', 'Ncd'),
 ('？', 'QUESTIONCATEGORY'),
 ('6', 'Neu'),
 ('張', 'Nf'),
 ('圖', 'VF'),
 ('一目瞭然', 'VH'),
 ('！', 'EXCLAMATIONCATEGORY')]

# Group mean vector

In [30]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(train_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)
group_mean_vector

{'體育': array([0.04      , 0.01333333, 0.00333333, ..., 0.        , 0.        ,
        0.        ]),
 '財經': array([0., 0., 0., ..., 0., 0., 0.]),
 '科技': array([0., 0., 0., ..., 0., 0., 0.]),
 '旅遊': array([0., 0., 0., ..., 0., 0., 0.]),
 '農業': array([0., 0., 0., ..., 0., 0., 0.]),
 '遊戲': array([0.        , 0.        , 0.        , ..., 0.00333333, 0.00333333,
        0.00333333])}

# Group mean vector: 測試

In [31]:
classification = {news_class: [] for news_class in all_news_class}
for index, pairs in sorted(test_title_cuts.items()):
    vector = get_bow_vector_with_selection(pairs, word2index)
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [32]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 73, '遊戲': 8, '財經': 6, '旅遊': 6, '農業': 6, '科技': 3})
predict 財經 :  Counter({'財經': 67, '科技': 18, '體育': 6, '旅遊': 6, '遊戲': 6, '農業': 5})
predict 科技 :  Counter({'科技': 62, '財經': 13, '體育': 10, '農業': 6, '遊戲': 6, '旅遊': 2})
predict 旅遊 :  Counter({'旅遊': 70, '農業': 11, '財經': 6, '科技': 3, '遊戲': 2})
predict 農業 :  Counter({'農業': 70, '旅遊': 9, '科技': 6, '體育': 5, '財經': 4, '遊戲': 3})
predict 遊戲 :  Counter({'遊戲': 75, '科技': 7, '旅遊': 5, '體育': 4, '財經': 4, '農業': 1})
